# Qwen Image Edit 2509 - Édition Avancée d'Images

**Module :** 02-Images-Advanced  
**Niveau :** Intermédiaire/Avancé  
**Durée estimée :** 45 minutes  

## Introduction

Ce notebook explore les capacités avancées de **Qwen-Image-Edit 2509** (version Septembre 2025), un modèle d'édition d'images de pointe intégré via ComfyUI. Par rapport au notebook d'introduction (01-5), nous abordons ici :

- **Édition précise de texte** dans les images
- **Inpainting avancé** avec masques personnalisés
- **Workflows multi-étapes** pour des transformations complexes
- **Batch processing** pour l'efficacité
- **Analyse comparative** des paramètres

### Architecture Qwen-Image-Edit 2509

```
┌─────────────────────────────────────────────────────────┐
│              Qwen-Image-Edit 2509 Pipeline              │
├─────────────────────────────────────────────────────────┤
│  Image Input → Qwen2.5-VL Encoder → Diffusion Model    │
│       ↓              ↓                    ↓             │
│  [Tokenizer]    [16-ch VAE]        [UNet 1024²]        │
│       ↓              ↓                    ↓             │
│   Text Prompt → Cross-Attention → Latent Space         │
│                                        ↓                │
│                               VAE Decode → Output      │
└─────────────────────────────────────────────────────────┘
```

## Prérequis

- Module 00-GenAI-Environment complété
- Service `comfyui-qwen` actif (`docker compose up -d`)
- Notebook 01-5-Qwen-Image-Edit terminé (concepts de base)

In [ ]:
# =============================================================================
# 1. CONFIGURATION ET IMPORTS
# =============================================================================

import os
import sys
import json
import uuid
import time
import base64
import requests
from io import BytesIO
from pathlib import Path
from datetime import datetime
from typing import Optional, Dict, List, Tuple, Any

import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

# Chargement variables d'environnement
# Recherche du .env en remontant l'arborescence (plus robuste que chemins relatifs)
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(5):  # Remonter jusqu'a 5 niveaux
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    # Fallback: essayer des chemins relatifs connus
    for fallback in ["../../.env", "../.env", "../00-GenAI-Environment/.env"]:
        if Path(fallback).exists():
            load_dotenv(fallback)
            found_env = True
            break

# Configuration ComfyUI
# URL par defaut: service myia.io pour etudiants
COMFYUI_URL = os.getenv("COMFYUI_API_URL", "https://qwen-image-edit.myia.io")
# Support des deux noms de variable pour le token
COMFYUI_TOKEN = os.getenv("COMFYUI_AUTH_TOKEN") or os.getenv("COMFYUI_API_TOKEN")
CLIENT_ID = str(uuid.uuid4())

# Validation
if not COMFYUI_TOKEN:
    raise ValueError("COMFYUI_AUTH_TOKEN ou COMFYUI_API_TOKEN manquant dans .env")

print("Qwen-Image-Edit 2509 - Edition Avancee")
print(f"\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"API URL: {COMFYUI_URL}")
print(f"Token: {'Configure' if COMFYUI_TOKEN else 'Manquant'}")
if found_env:
    print(f".env: charge")

In [ ]:
# =============================================================================
# 2. CLIENT COMFYUI AVANCÉ
# =============================================================================

class QwenImageEditClient:
    """
    Client avancé pour Qwen-Image-Edit via ComfyUI.
    Supporte l'authentification, le batch processing et les workflows multi-étapes.
    """
    
    def __init__(self, base_url: str = COMFYUI_URL, auth_token: str = COMFYUI_TOKEN):
        self.base_url = base_url.rstrip('/')
        self.client_id = str(uuid.uuid4())
        self.session = requests.Session()
        
        if auth_token:
            self.session.headers.update({"Authorization": f"Bearer {auth_token}"})
    
    def check_health(self) -> Dict[str, Any]:
        """Vérifie la santé du service ComfyUI."""
        try:
            resp = self.session.get(f"{self.base_url}/system_stats", timeout=10)
            if resp.status_code == 200:
                stats = resp.json()
                return {
                    "status": "healthy",
                    "vram_free": stats.get("devices", [{}])[0].get("vram_free", 0) / 1e9,
                    "vram_total": stats.get("devices", [{}])[0].get("vram_total", 0) / 1e9
                }
            return {"status": "error", "code": resp.status_code}
        except Exception as e:
            return {"status": "unreachable", "error": str(e)}
    
    def upload_image(self, image: Image.Image, name: str = "input.png") -> str:
        """Upload une image vers ComfyUI."""
        buffer = BytesIO()
        image.save(buffer, format="PNG")
        buffer.seek(0)
        
        files = {"image": (name, buffer, "image/png")}
        data = {"overwrite": "true"}
        
        resp = self.session.post(f"{self.base_url}/upload/image", files=files, data=data)
        if resp.status_code == 200:
            return resp.json().get("name", name)
        raise Exception(f"Upload failed: {resp.text}")
    
    def upload_mask(self, mask: Image.Image, name: str = "mask.png") -> str:
        """Upload un masque pour l'inpainting."""
        # Convertir en grayscale si nécessaire
        if mask.mode != 'L':
            mask = mask.convert('L')
        return self.upload_image(mask, name)
    
    def queue_workflow(self, workflow: Dict) -> str:
        """Soumet un workflow et retourne le prompt_id."""
        payload = {"prompt": workflow, "client_id": self.client_id}
        resp = self.session.post(f"{self.base_url}/prompt", json=payload)
        
        if resp.status_code != 200:
            raise Exception(f"Queue failed: {resp.text}")
        return resp.json()["prompt_id"]
    
    def wait_for_completion(self, prompt_id: str, timeout: int = 120) -> Dict:
        """Attend la fin d'un workflow avec timeout."""
        start = time.time()
        while time.time() - start < timeout:
            resp = self.session.get(f"{self.base_url}/history/{prompt_id}")
            if resp.status_code == 200:
                history = resp.json()
                if prompt_id in history:
                    return history[prompt_id]
            time.sleep(1)
        raise TimeoutError(f"Workflow {prompt_id} timeout after {timeout}s")
    
    def get_image(self, filename: str, subfolder: str = "", img_type: str = "output") -> Image.Image:
        """Récupère une image générée."""
        params = {"filename": filename, "subfolder": subfolder, "type": img_type}
        resp = self.session.get(f"{self.base_url}/view", params=params)
        if resp.status_code == 200:
            return Image.open(BytesIO(resp.content))
        raise Exception(f"Failed to get image: {resp.text}")
    
    def execute_and_get_images(self, workflow: Dict, output_node: str = "9", 
                                timeout: int = 120, verbose: bool = True) -> List[Image.Image]:
        """Exécute un workflow et retourne les images générées."""
        if verbose:
            print("🚀 Soumission du workflow...")
        
        prompt_id = self.queue_workflow(workflow)
        if verbose:
            print(f"📋 ID: {prompt_id}")
            print("⏳ Génération en cours...", end="", flush=True)
        
        result = self.wait_for_completion(prompt_id, timeout)
        if verbose:
            print(" ✅")
        
        # Extraire les images
        images = []
        if "outputs" in result and output_node in result["outputs"]:
            for img_data in result["outputs"][output_node].get("images", []):
                img = self.get_image(
                    img_data["filename"],
                    img_data.get("subfolder", ""),
                    img_data.get("type", "output")
                )
                images.append(img)
        
        if verbose:
            print(f"🖼️ {len(images)} image(s) récupérée(s)")
        return images

# Instanciation du client
client = QwenImageEditClient()

# Test de connexion
health = client.check_health()
print(f"\n🏥 État du service: {health['status']}")
if health['status'] == 'healthy':
    print(f"   VRAM: {health['vram_free']:.1f} / {health['vram_total']:.1f} GB libre")

In [ ]:
# =============================================================================
# 3. WORKFLOWS QWEN-IMAGE-EDIT 2509 - ARCHITECTURE PHASE 29
# =============================================================================
# Ces workflows utilisent l'architecture native ComfyUI validee Phase 29
# avec les modeles FP8 officiels Comfy-Org

def create_text2img_workflow(prompt: str, 
                              width: int = 1024, height: int = 1024,
                              steps: int = 20, cfg: float = 1.0,
                              seed: int = None) -> Dict:
    """
    Workflow Text-to-Image avec Qwen-Image-Edit 2509.
    
    Architecture Phase 29 validee:
    - VAELoader + CLIPLoader + UNETLoader (modeles separes)
    - ModelSamplingAuraFlow (shift=3.0) + CFGNorm (strength=1.0)
    - TextEncodeQwenImageEdit (encodeur natif Qwen)
    - KSampler avec scheduler=beta, sampler=euler, cfg=1.0
    
    Args:
        prompt: Description de l'image a generer
        width/height: Dimensions (multiples de 32, defaut 1024)
        steps: Nombre d'etapes de diffusion (20 recommande)
        cfg: Guidance scale (1.0 recommande pour Qwen avec CFGNorm)
        seed: Graine pour reproductibilite
    """
    if seed is None:
        seed = np.random.randint(0, 2**32)
    
    return {
        # Chargement des modeles
        "1": {
            "class_type": "VAELoader",
            "inputs": {"vae_name": "qwen_image_vae.safetensors"}
        },
        "2": {
            "class_type": "CLIPLoader",
            "inputs": {
                "clip_name": "qwen_2.5_vl_7b_fp8_scaled.safetensors",
                "type": "sd3"
            }
        },
        "3": {
            "class_type": "UNETLoader",
            "inputs": {
                "unet_name": "qwen_image_edit_2509_fp8_e4m3fn.safetensors",
                "weight_dtype": "fp8_e4m3fn"
            }
        },
        # Configuration modele pour Qwen
        "4": {
            "class_type": "ModelSamplingAuraFlow",
            "inputs": {"model": ["3", 0], "shift": 3.0}
        },
        "5": {
            "class_type": "CFGNorm",
            "inputs": {"model": ["4", 0], "strength": 1.0}
        },
        # Encodage du prompt avec TextEncodeQwenImageEdit
        "6": {
            "class_type": "TextEncodeQwenImageEdit",
            "inputs": {
                "clip": ["2", 0],
                "prompt": prompt,
                "vae": ["1", 0]
            }
        },
        # Conditioning negatif (vide pour Qwen)
        "7": {
            "class_type": "ConditioningZeroOut",
            "inputs": {"conditioning": ["6", 0]}
        },
        # Latent vide (16 canaux pour Qwen)
        "8": {
            "class_type": "EmptySD3LatentImage",
            "inputs": {"width": width, "height": height, "batch_size": 1}
        },
        # Sampling
        "9": {
            "class_type": "KSampler",
            "inputs": {
                "model": ["5", 0],
                "positive": ["6", 0],
                "negative": ["7", 0],
                "latent_image": ["8", 0],
                "seed": seed,
                "steps": steps,
                "cfg": cfg,
                "sampler_name": "euler",
                "scheduler": "beta",
                "denoise": 1.0
            }
        },
        # Decodage VAE
        "10": {
            "class_type": "VAEDecode",
            "inputs": {"samples": ["9", 0], "vae": ["1", 0]}
        },
        # Sauvegarde
        "11": {
            "class_type": "SaveImage",
            "inputs": {"filename_prefix": "Qwen2509_t2i", "images": ["10", 0]}
        }
    }


def create_img2img_workflow(image_name: str, prompt: str, 
                            denoise: float = 0.7, steps: int = 20,
                            cfg: float = 1.0, seed: int = None) -> Dict:
    """
    Workflow Image-to-Image pour l'edition avec Qwen.
    
    Note: Qwen-Image-Edit 2509 est optimise pour l'edition d'images.
    Le parametre denoise controle la force de l'edition:
    - 0.3-0.5: Ajustements subtils
    - 0.5-0.7: Modifications moderees  
    - 0.7-0.9: Transformations significatives
    
    Args:
        image_name: Nom du fichier image uploade
        prompt: Instructions d'edition
        denoise: Force de l'edition (0.0=rien, 1.0=regeneration complete)
    """
    if seed is None:
        seed = np.random.randint(0, 2**32)
    
    return {
        # Chargement image source
        "1": {
            "class_type": "LoadImage",
            "inputs": {"image": image_name}
        },
        # Chargement des modeles
        "2": {
            "class_type": "VAELoader",
            "inputs": {"vae_name": "qwen_image_vae.safetensors"}
        },
        "3": {
            "class_type": "CLIPLoader",
            "inputs": {
                "clip_name": "qwen_2.5_vl_7b_fp8_scaled.safetensors",
                "type": "sd3"
            }
        },
        "4": {
            "class_type": "UNETLoader",
            "inputs": {
                "unet_name": "qwen_image_edit_2509_fp8_e4m3fn.safetensors",
                "weight_dtype": "fp8_e4m3fn"
            }
        },
        # Configuration modele
        "5": {
            "class_type": "ModelSamplingAuraFlow",
            "inputs": {"model": ["4", 0], "shift": 3.0}
        },
        "6": {
            "class_type": "CFGNorm",
            "inputs": {"model": ["5", 0], "strength": 1.0}
        },
        # Encodage VAE de l'image source
        "7": {
            "class_type": "VAEEncode",
            "inputs": {"pixels": ["1", 0], "vae": ["2", 0]}
        },
        # Encodage du prompt
        "8": {
            "class_type": "TextEncodeQwenImageEdit",
            "inputs": {
                "clip": ["3", 0],
                "prompt": prompt,
                "vae": ["2", 0]
            }
        },
        # Conditioning negatif
        "9": {
            "class_type": "ConditioningZeroOut",
            "inputs": {"conditioning": ["8", 0]}
        },
        # Sampling
        "10": {
            "class_type": "KSampler",
            "inputs": {
                "model": ["6", 0],
                "positive": ["8", 0],
                "negative": ["9", 0],
                "latent_image": ["7", 0],
                "seed": seed,
                "steps": steps,
                "cfg": cfg,
                "sampler_name": "euler",
                "scheduler": "beta",
                "denoise": denoise
            }
        },
        # Decodage
        "11": {
            "class_type": "VAEDecode",
            "inputs": {"samples": ["10", 0], "vae": ["2", 0]}
        },
        # Sauvegarde
        "12": {
            "class_type": "SaveImage",
            "inputs": {"filename_prefix": "Qwen2509_i2i", "images": ["11", 0]}
        }
    }


def create_inpaint_workflow(image_name: str, mask_name: str, prompt: str,
                            denoise: float = 0.9, steps: int = 25,
                            cfg: float = 1.0, seed: int = None) -> Dict:
    """
    Workflow Inpainting pour Qwen-Image-Edit 2509.

    Note: L'inpainting Qwen utilise SetLatentNoiseMask pour appliquer
    le masque sur le latent encode. Le masque definit les zones a regenerer.

    Args:
        image_name: Nom du fichier image uploade
        mask_name: Nom du fichier masque (blanc = zone a modifier)
        prompt: Description de ce qui doit remplacer la zone masquee
        denoise: Force de regeneration (0.8-1.0 recommande pour inpaint)
    """
    if seed is None:
        seed = np.random.randint(0, 2**32)

    return {
        # Chargement image et masque
        "1": {
            "class_type": "LoadImage",
            "inputs": {"image": image_name}
        },
        "2": {
            "class_type": "LoadImage",
            "inputs": {"image": mask_name}
        },
        # Chargement des modeles
        "3": {
            "class_type": "VAELoader",
            "inputs": {"vae_name": "qwen_image_vae.safetensors"}
        },
        "4": {
            "class_type": "CLIPLoader",
            "inputs": {
                "clip_name": "qwen_2.5_vl_7b_fp8_scaled.safetensors",
                "type": "sd3"
            }
        },
        "5": {
            "class_type": "UNETLoader",
            "inputs": {
                "unet_name": "qwen_image_edit_2509_fp8_e4m3fn.safetensors",
                "weight_dtype": "fp8_e4m3fn"
            }
        },
        # Configuration modele
        "6": {
            "class_type": "ModelSamplingAuraFlow",
            "inputs": {"model": ["5", 0], "shift": 3.0}
        },
        "7": {
            "class_type": "CFGNorm",
            "inputs": {"model": ["6", 0], "strength": 1.0}
        },
        # Encodage VAE de l'image source
        "8": {
            "class_type": "VAEEncode",
            "inputs": {"pixels": ["1", 0], "vae": ["3", 0]}
        },
        # Application du masque sur le latent
        "9": {
            "class_type": "SetLatentNoiseMask",
            "inputs": {
                "samples": ["8", 0],
                "mask": ["2", 0]
            }
        },
        # Encodage du prompt
        "10": {
            "class_type": "TextEncodeQwenImageEdit",
            "inputs": {
                "clip": ["4", 0],
                "prompt": prompt,
                "vae": ["3", 0]
            }
        },
        # Conditioning negatif
        "11": {
            "class_type": "ConditioningZeroOut",
            "inputs": {"conditioning": ["10", 0]}
        },
        # Sampling avec masque
        "12": {
            "class_type": "KSampler",
            "inputs": {
                "model": ["7", 0],
                "positive": ["10", 0],
                "negative": ["11", 0],
                "latent_image": ["9", 0],
                "seed": seed,
                "steps": steps,
                "cfg": cfg,
                "sampler_name": "euler",
                "scheduler": "beta",
                "denoise": denoise
            }
        },
        # Decodage
        "13": {
            "class_type": "VAEDecode",
            "inputs": {"samples": ["12", 0], "vae": ["3", 0]}
        },
        # Sauvegarde
        "14": {
            "class_type": "SaveImage",
            "inputs": {"filename_prefix": "Qwen2509_inpaint", "images": ["13", 0]}
        }
    }


print("Workflows Qwen-Image-Edit 2509 definis (Architecture Phase 29)")
print("   - create_text2img_workflow() -> node output: '11'")
print("   - create_img2img_workflow() -> node output: '12'")
print("   - create_inpaint_workflow() -> node output: '14'")
print("\nNotes importantes:")
print("   - CFG=1.0 recommande (CFGNorm gere l'amplification)")
print("   - Scheduler 'beta' optimise pour Qwen")
print("   - 20 steps suffisent pour de bons resultats")

## 4. Génération Text-to-Image

Commençons par générer une image de base que nous éditerons ensuite.

In [ ]:
# =============================================================================
# 4. TEXT-TO-IMAGE: Creation d'une image de base
# =============================================================================

# Prompt creatif pour une scene editable
prompt_base = """
A cozy coffee shop interior, wooden tables, warm lighting,
large window with rain outside, vintage aesthetic,
empty cup on table, potted plant, high quality photography
""".strip()

# Creer le workflow (Architecture Phase 29)
workflow_t2i = create_text2img_workflow(
    prompt=prompt_base,
    width=1024,
    height=768,
    steps=20,      # 20 steps optimaux pour Qwen
    cfg=1.0,       # CFG=1.0 avec CFGNorm
    seed=42        # Fixe pour reproductibilite
)

# Executer (output_node="11" pour le nouveau workflow)
print("\nGeneration de l'image de base...")
print(f"Prompt: {prompt_base[:80]}...")

images_t2i = client.execute_and_get_images(workflow_t2i, output_node="11")

if images_t2i:
    base_image = images_t2i[0]
    
    # Affichage
    plt.figure(figsize=(12, 9))
    plt.imshow(base_image)
    plt.title("Image de Base - Coffee Shop (Qwen 2509)", fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"\nDimensions: {base_image.size}")
else:
    print("Erreur de generation - verifiez que les modeles Qwen sont presents")
    print("Modeles requis dans ComfyUI/models/:")
    print("  - vae/qwen_image_vae.safetensors")
    print("  - text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors")
    print("  - diffusion_models/qwen_image_edit_2509_fp8_e4m3fn.safetensors")

## 5. Édition Image-to-Image

Explorons différents niveaux de `denoise` pour comprendre son impact sur l'édition.

In [ ]:
# =============================================================================
# 5. IMAGE-TO-IMAGE: Analyse comparative du parametre denoise
# =============================================================================

# Upload de l'image de base
if 'base_image' in dir():
    uploaded_name = client.upload_image(base_image, "base_coffee_shop.png")
    print(f"Image uploadee: {uploaded_name}")
    
    # Test avec differents niveaux de denoise
    denoise_levels = [0.3, 0.5, 0.7, 0.9]
    edit_prompt = "Same scene but with snow falling outside the window, winter atmosphere"
    
    results = []
    seed_fixed = 12345  # Meme seed pour comparer
    
    print(f"\nEdition avec prompt: '{edit_prompt}'")
    print("\nComparaison des niveaux de denoise:")
    
    for denoise in denoise_levels:
        print(f"\n--- Denoise = {denoise} ---")
        
        workflow = create_img2img_workflow(
            image_name=uploaded_name,
            prompt=edit_prompt,
            denoise=denoise,
            steps=20,
            cfg=1.0,
            seed=seed_fixed
        )
        
        # output_node="12" pour img2img
        images = client.execute_and_get_images(workflow, output_node="12", verbose=False)
        if images:
            results.append((denoise, images[0]))
            print(f"   Genere")
    
    # Affichage comparatif
    if results:
        fig, axes = plt.subplots(1, len(results) + 1, figsize=(20, 5))
        
        # Image originale
        axes[0].imshow(base_image)
        axes[0].set_title("Original", fontsize=12)
        axes[0].axis('off')
        
        # Resultats
        for i, (denoise, img) in enumerate(results):
            axes[i+1].imshow(img)
            axes[i+1].set_title(f"Denoise = {denoise}", fontsize=12)
            axes[i+1].axis('off')
        
        plt.suptitle("Impact du parametre Denoise sur l'edition", fontsize=14)
        plt.tight_layout()
        plt.show()
        
        print("\nObservations:")
        print("   0.3: Changements subtils, structure tres preservee")
        print("   0.5: Modifications visibles, bonne balance")
        print("   0.7: Transformations significatives")
        print("   0.9: Quasi-regeneration, peu de l'original conserve")
else:
    print("Executez d'abord la cellule Text-to-Image")

## 6. Inpainting Avancé avec Masque Personnalisé

L'inpainting permet de modifier uniquement certaines zones de l'image. Nous allons créer un masque programmatique pour remplacer un élément spécifique.

In [ ]:
# =============================================================================
# 6. INPAINTING: Édition localisée avec masque
# =============================================================================

def create_rectangular_mask(width: int, height: int, 
                            x1: int, y1: int, x2: int, y2: int,
                            feather: int = 10) -> Image.Image:
    """
    Crée un masque rectangulaire avec bords adoucis.
    
    Args:
        width, height: Dimensions du masque
        x1, y1, x2, y2: Coordonnées du rectangle (zone à modifier)
        feather: Adoucissement des bords en pixels
    
    Returns:
        Image grayscale (blanc = zone à modifier)
    """
    mask = Image.new('L', (width, height), 0)  # Noir = préserver
    draw = ImageDraw.Draw(mask)
    draw.rectangle([x1, y1, x2, y2], fill=255)  # Blanc = modifier
    
    # Adoucissement optionnel (blur simple)
    if feather > 0:
        from PIL import ImageFilter
        mask = mask.filter(ImageFilter.GaussianBlur(radius=feather))
    
    return mask


def create_circular_mask(width: int, height: int,
                         cx: int, cy: int, radius: int,
                         feather: int = 15) -> Image.Image:
    """
    Crée un masque circulaire pour l'inpainting.
    """
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse([cx-radius, cy-radius, cx+radius, cy+radius], fill=255)
    
    if feather > 0:
        from PIL import ImageFilter
        mask = mask.filter(ImageFilter.GaussianBlur(radius=feather))
    
    return mask


# Exemple: Remplacer la tasse sur la table
if 'base_image' in dir():
    w, h = base_image.size
    
    # Créer un masque pour le centre-bas de l'image (où la table/tasse serait)
    mask = create_rectangular_mask(
        w, h,
        x1=int(w*0.35), y1=int(h*0.55),
        x2=int(w*0.65), y2=int(h*0.85),
        feather=20
    )
    
    # Upload du masque
    mask_name = client.upload_mask(mask, "edit_mask.png")
    print(f"✅ Masque uploadé: {mask_name}")
    
    # Visualisation du masque
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(base_image)
    axes[0].set_title("Image Originale")
    axes[0].axis('off')
    
    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title("Masque (blanc = zone à modifier)")
    axes[1].axis('off')
    
    # Overlay
    overlay = base_image.copy().convert('RGBA')
    mask_rgba = Image.new('RGBA', overlay.size, (255, 0, 0, 0))
    mask_draw = ImageDraw.Draw(mask_rgba)
    # Convertir le masque en overlay rouge semi-transparent
    mask_array = np.array(mask)
    red_overlay = np.zeros((h, w, 4), dtype=np.uint8)
    red_overlay[:,:,0] = 255  # Rouge
    red_overlay[:,:,3] = (mask_array * 0.5).astype(np.uint8)  # Alpha
    overlay_img = Image.alpha_composite(overlay, Image.fromarray(red_overlay))
    
    axes[2].imshow(overlay_img)
    axes[2].set_title("Zone d'édition (rouge)")
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Image de base non disponible")

In [ ]:
# =============================================================================
# 6b. EXECUTION DE L'INPAINTING
# =============================================================================

if 'uploaded_name' in dir() and 'mask_name' in dir():
    # Prompt pour la zone masquee
    inpaint_prompt = "A beautiful laptop with glowing screen, modern design, on wooden table"
    
    print(f"\nInpainting: '{inpaint_prompt}'")
    
    workflow_inpaint = create_inpaint_workflow(
        image_name=uploaded_name,
        mask_name=mask_name,
        prompt=inpaint_prompt,
        denoise=0.95,  # Haut pour remplacer completement
        steps=25,
        cfg=1.0,       # CFG=1.0 avec CFGNorm (Phase 29)
        seed=99999
    )
    
    # output_node="14" pour inpainting workflow
    images_inpaint = client.execute_and_get_images(workflow_inpaint, output_node="14")
    
    if images_inpaint:
        inpainted_image = images_inpaint[0]
        
        # Comparaison avant/apres
        fig, axes = plt.subplots(1, 2, figsize=(14, 7))
        
        axes[0].imshow(base_image)
        axes[0].set_title("Avant Inpainting", fontsize=14)
        axes[0].axis('off')
        
        axes[1].imshow(inpainted_image)
        axes[1].set_title("Apres Inpainting", fontsize=14)
        axes[1].axis('off')
        
        plt.suptitle(f"Inpainting: '{inpaint_prompt}'", fontsize=12)
        plt.tight_layout()
        plt.show()
else:
    print("Prerequis manquants (image ou masque)")

## 7. Batch Processing: Génération Multiple

Pour l'efficacité, générons plusieurs variations en parallèle avec différents prompts.

In [ ]:
# =============================================================================
# 7. BATCH PROCESSING: Variations multiples
# =============================================================================

def batch_generate(prompts: List[str], base_seed: int = 1000, **kwargs) -> List[Tuple[str, Image.Image]]:
    """
    Genere plusieurs images a partir d'une liste de prompts.
    
    Args:
        prompts: Liste de descriptions
        base_seed: Seed de depart (incremente pour chaque image)
        **kwargs: Arguments passes a create_text2img_workflow
    
    Returns:
        Liste de tuples (prompt, image)
    """
    results = []
    
    for i, prompt in enumerate(prompts):
        print(f"\n[{i+1}/{len(prompts)}] Generation...")
        print(f"   Prompt: {prompt[:60]}...")
        
        workflow = create_text2img_workflow(
            prompt=prompt,
            seed=base_seed + i,
            **kwargs
        )
        
        # output_node="11" pour t2i
        images = client.execute_and_get_images(workflow, output_node="11", verbose=False)
        if images:
            results.append((prompt, images[0]))
            print(f"   Succes")
        else:
            print(f"   Echec")
    
    return results


# Generation de variations thematiques
variation_prompts = [
    "A futuristic cityscape at sunset, flying cars, neon lights, cyberpunk style",
    "An ancient Japanese temple in autumn, red maple leaves, misty mountains",
    "An underwater coral reef, tropical fish, sunlight rays, crystal clear water",
    "A cozy library interior, tall bookshelves, reading nook, warm lamp light"
]

print("\nBatch Generation - 4 Themes")
print("=" * 40)

batch_results = batch_generate(
    variation_prompts,
    base_seed=2024,
    width=768,
    height=768,
    steps=20,
    cfg=1.0  # CFG=1.0 avec CFGNorm
)

# Affichage grille
if batch_results:
    n = len(batch_results)
    cols = 2
    rows = (n + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, (prompt, img) in enumerate(batch_results):
        axes[i].imshow(img)
        # Titre court
        short_title = prompt.split(',')[0][:40]
        axes[i].set_title(short_title, fontsize=10)
        axes[i].axis('off')
    
    # Masquer les axes vides
    for i in range(len(batch_results), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle("Batch Generation - Variations Thematiques", fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"\n{len(batch_results)} images generees avec succes")

## 8. Analyse Comparative: CFG Scale

Le paramètre `cfg` (Classifier-Free Guidance) contrôle l'adhérence au prompt. Explorons son impact.

In [ ]:
# =============================================================================
# 8. ANALYSE CFG: Impact du Guidance Scale avec CFGNorm
# =============================================================================
# NOTE IMPORTANTE (Phase 29):
# L'architecture Qwen utilise CFGNorm qui normalise le guidance.
# Avec CFGNorm, CFG=1.0 est optimal car la normalisation gere l'amplification.
# Cette cellule explore differentes valeurs CFG a titre educatif pour
# comprendre le comportement du modele.

test_prompt = "A majestic dragon breathing fire, fantasy art, highly detailed scales, dramatic lighting"

# Valeurs CFG a tester (1.0 est recommande avec CFGNorm)
cfg_values = [1.0, 2.0, 4.0, 7.0]
fixed_seed = 7777

print(f"\nAnalyse CFG Scale (avec CFGNorm)")
print(f"Prompt: '{test_prompt[:50]}...'")
print(f"Valeurs testees: {cfg_values}")
print("\nNote: CFG=1.0 est recommande avec CFGNorm (architecture Phase 29)")

cfg_results = []

for cfg in cfg_values:
    print(f"\n--- CFG = {cfg} ---")
    
    workflow = create_text2img_workflow(
        prompt=test_prompt,
        width=768,
        height=768,
        steps=20,
        cfg=cfg,
        seed=fixed_seed
    )
    
    # output_node="11" pour t2i
    images = client.execute_and_get_images(workflow, output_node="11", verbose=False)
    if images:
        cfg_results.append((cfg, images[0]))
        print(f"   Genere")

# Affichage comparatif
if cfg_results:
    fig, axes = plt.subplots(1, len(cfg_results), figsize=(16, 5))
    
    for i, (cfg, img) in enumerate(cfg_results):
        axes[i].imshow(img)
        title = f"CFG = {cfg}"
        if cfg == 1.0:
            title += " (recommande)"
        axes[i].set_title(title, fontsize=12)
        axes[i].axis('off')
    
    plt.suptitle("Impact du CFG avec CFGNorm (Architecture Phase 29)", fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print("\nObservations CFG avec CFGNorm:")
    print("   1.0: Valeur optimale - CFGNorm gere l'amplification automatiquement")
    print("   2.0: Leger renforcement du prompt")
    print("   4.0: Adherence plus forte, details accentues")
    print("   7.0: Peut introduire des artefacts avec CFGNorm")
    print("\nRecommandation: Utiliser CFG=1.0 avec l'architecture Phase 29")

## 9. Exercices Pratiques

### Exercice 1: Édition de Style
Prenez l'image de base du coffee shop et appliquez différents styles artistiques (impressionniste, anime, réaliste) en utilisant img2img avec un denoise de 0.6.

### Exercice 2: Inpainting Créatif
Créez un masque circulaire au centre de l'image et remplacez cette zone par un personnage de votre choix.

### Exercice 3: Exploration des Schedulers
Modifiez le workflow pour tester différents schedulers (`normal`, `karras`, `exponential`) et comparez les résultats.

In [ ]:
# =============================================================================
# 9. ESPACE D'EXERCICES
# =============================================================================

# Exercice 1: Style Transfer
# Décommentez et complétez:

# style_prompts = [
#     "Same scene, impressionist painting style, visible brushstrokes",
#     "Same scene, anime style, vibrant colors, Studio Ghibli",
#     "Same scene, photorealistic, DSLR quality, 8k resolution"
# ]
# 
# for style in style_prompts:
#     workflow = create_img2img_workflow(
#         image_name=uploaded_name,
#         prompt=style,
#         denoise=0.6
#     )
#     # ... générer et afficher

print("📝 Espace d'exercices - Décommentez le code ci-dessus pour commencer")

## 10. Recapitulatif et Points Cles

### Architecture Phase 29 - Qwen-Image-Edit 2509

Cette architecture utilise les composants natifs ComfyUI suivants :

| Composant | Node ComfyUI | Fichier Modele |
|-----------|--------------|----------------|
| VAE | VAELoader | qwen_image_vae.safetensors (243MB) |
| Text Encoder | CLIPLoader (type=sd3) | qwen_2.5_vl_7b_fp8_scaled.safetensors (8.8GB) |
| Diffusion | UNETLoader (fp8_e4m3fn) | qwen_image_edit_2509_fp8_e4m3fn.safetensors (20GB) |

### Parametres Essentiels (Architecture Phase 29)

| Parametre | Plage | Recommande | Impact |
|-----------|-------|------------|--------|
| `steps` | 15-30 | 20 | Qualite vs. Vitesse |
| `cfg` | 1-4 | **1.0** | CFGNorm gere l'amplification |
| `denoise` | 0-1 | 0.5-0.7 | Force de l'edition |
| `shift` | 2-4 | 3.0 | ModelSamplingAuraFlow |
| `scheduler` | - | **beta** | Optimise pour Qwen |
| `sampler` | - | **euler** | Stable et rapide |

### Bonnes Pratiques

1. **Toujours utiliser CFG=1.0** avec l'architecture Phase 29 (CFGNorm normalise automatiquement)
2. **Scheduler 'beta'** est optimise pour les modeles Qwen
3. **20 steps** suffisent pour de bons resultats (qualite/vitesse optimal)
4. **Pour l'inpainting, denoise >= 0.8** pour un remplacement complet
5. **Utiliser seed fixe** pour comparer les parametres et reproduire les resultats

### Nodes Requis

```
TextEncodeQwenImageEdit   # Encodeur texte natif Qwen
ModelSamplingAuraFlow     # Configuration sampling (shift=3.0)
CFGNorm                   # Normalisation CFG (strength=1.0)
EmptySD3LatentImage       # Latent 16 canaux pour Qwen
ConditioningZeroOut       # Conditioning negatif vide
```

### Ressources

- [Documentation ComfyUI](https://docs.comfy.org/)
- [Qwen-VL Papers](https://arxiv.org/abs/2308.12966)
- [Modeles Comfy-Org](https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI)
- Notebook suivant: **02-2-FLUX-1-Advanced-Generation**

In [ ]:
# =============================================================================
# FIN DU NOTEBOOK
# =============================================================================

print("\n" + "="*60)
print("   ✅ Notebook Qwen-Image-Edit 2509 Complété")
print("="*60)
print(f"\n📅 Terminé: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n📚 Concepts couverts:")
print("   • Text-to-Image avec Qwen 2509")
print("   • Image-to-Image et analyse du denoise")
print("   • Inpainting avec masques personnalisés")
print("   • Batch processing")
print("   • Analyse comparative CFG")
print("\n➡️  Prochain notebook: 02-2-FLUX-1-Advanced-Generation.ipynb")